In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

import numpy as np


# Create a Spark session
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()
sc = SparkContext.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")



23/12/06 17:05:43 WARN Utils: Your hostname, MacBook-Pro-di-Francesco-4.local resolves to a loopback address: 127.0.0.1; using 192.168.1.181 instead (on interface en0)
23/12/06 17:05:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 17:05:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

def readFile(filename):
    # Initialize Spark context
    #spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

    #sc = spark.sparkContext

    # Read the file into an RDD
    # Each line in the file becomes one record in the RDD
    lines = sc.textFile(filename)

    # Process each line in the RDD
    data = lines.map(lambda line: line.split(','))\
                .map(lambda elem: ([float(x) for x in elem[:-1]], int(elem[-1])))

    return data

In [3]:
#Testing of the read function
filename = 'data/botnet_tot_syn_l.csv'
RDD_Xy = readFile(filename)

# Printing first 5 rows of the RDD
for row in RDD_Xy.take(5):
    print(row)

([9.012784269851089, 1672.9999766891833, 21.99998846107087, 0.9999997452701503, 61.99988768910407, 69.99980788311223, 13.000000232993186, 2.9999999785934133, 199.00000192395984, 2468369573.0148935, 2468372549.224571], 1)
([3599.9990884099398, 48206.57583515873, 13362.999847123127, 1.0000019205086303, 262.99924185046257, 82.99988759205826, 13.999945225127506, 5.000000032871429, 216.99999612378573, 1539044199.5873349, 2468368394.7593513], 0)
([0.0006999386757797765, 0.0009472844685660675, 53.000260213743786, 2.0000004196928174, 646242903.4069201, 82.99988759205826, 12.999999992936175, 2.999999985175615, 216.99999612378573, 2468369544.907854, 2503250078.1888514], 0)
([3599.9990884099398, 1718.7743566535646, 63377.68872854763, 1.0000019205086303, 926682.197305462, 543.5568914038613, 12.999999992936175, 5.000000032871429, 186.99999400676145, 405749204.8178947, 1125424493.3033822], 0)
([3471.3792185025277, 48024.1859162382, 0.00035088937602267833, 507008.9762772573, 262.99924185046257, 1000.

In [4]:
'''
def compute_stats(RDD):
    # Calculate sum and sum of squares for each feature
    sums = RDD.map(lambda x: x[0]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))])
    return sums
print(compute_stats(rdd))
'''

'\ndef compute_stats(RDD):\n    # Calculate sum and sum of squares for each feature\n    sums = RDD.map(lambda x: x[0]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))])\n    return sums\nprint(compute_stats(rdd))\n'

In [5]:
def compute_mean_stds(RDD):
    # Calculate sum and sum of squares for each feature
    sums = RDD.map(lambda x: x[0]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))]) 
    sum_of_squares = RDD.map(lambda x: [xi ** 2 for xi in x[0]]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))])
    count = RDD.count()

    # Compute mean and standard deviation for each feature
    means = [sums[i] / count for i in range(len(sums))]
    stds = [(sum_of_squares[i] / count - (means[i]) ** 2) ** 0.5 for i in range(len(means))]
    
    return means, stds



def normalize(RDD_Xy):
    # Compute means and standard deviations
    means, stds = compute_mean_stds(RDD_Xy)

    # Broadcast the means and standard deviations
    bc_means = sc.broadcast(means)
    bc_stds = sc.broadcast(stds)

    # Normalize each feature
    def normalize_features(features, means, stds):
        return [(features[i] - means[i]) / stds[i] if stds[i] != 0 else 0 for i in range(len(features))]
        #return [(features[i] - means[i]) / stds[i] for i in range(len(features))]

    normalized_RDD = RDD_Xy.map(lambda x: (normalize_features(x[0], bc_means.value, bc_stds.value), x[1]))

    return normalized_RDD



In [6]:
#Testing of the normalize function

normalized_RDD = normalize(RDD_Xy).cache()

# Print first 5 normalized rows
for row in normalized_RDD.take(5):
    print(row)


([-0.792409774778866, -0.8130937083307339, -0.4224507642218906, -0.4664697491193844, -0.5223929554497563, -0.35631956924945285, 0.7370102960327225, 0.5283496333423401, 0.8271779941619601, 0.4731661571170488, 0.15895172357621995], 1)
([1.443720404002351, 1.1163610034798641, 0.39127563577819524, -0.4664697491031832, -0.5223920961870128, -0.35631927462193325, 0.9263319360863808, 1.480120748267978, 1.0264183320400917, -0.8286954523619166, 0.15894853159524694], 0)
([-0.7980216574316705, -0.8824624555329105, -0.4205599209717913, -0.46646230111358483, 2.240264662197529, -0.35631927462193325, 0.7370102505822355, 0.5283496364747148, 1.0264183320400917, 0.4731661177428138, 0.185749013466657], 0)
([1.443720404002351, -0.8111957326975887, 3.441892018491165, -0.4664697491031832, -0.5184316978805608, -0.35630883678072567, 0.7370102505822355, 1.480120748267978, 0.6943509718073647, -2.416291452741655, -0.8728693208325027], 0)
([1.3636279869526706, 1.108798440707592, -0.42379261911074273, 3.30972605187

In [7]:
import numpy as np
from pyspark import SparkContext

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def linear_combination(features, weights, bias):
    return sum(weights[j] * features[j] for j in range(len(weights))) + bias

def compute_gradients_and_cost(record, weights, bias):
    xi, yi = record
    y_hat = sigmoid(linear_combination(xi, weights, bias))
    error = y_hat - yi
    dw = np.zeros(len(weights))
    for j in range(len(weights)):
        dw[j] = error * xi[j]
    db = error
    cost_contribution = -yi * np.log(y_hat) - (1 - yi) * np.log(1 - y_hat)
    return (dw, db, cost_contribution)

def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    np.random.seed(0) 
    feature_count = len(RDD_Xy.first()[0])
    w = np.random.rand(feature_count)
    b = np.random.rand()
    m = RDD_Xy.count()

    for n in range(iterations):
        bc_w = sc.broadcast(w)
        bc_b = sc.broadcast(b)

        gradients_and_cost = RDD_Xy.map(lambda x: compute_gradients_and_cost(x, bc_w.value, bc_b.value))
        dw, db, total_cost = gradients_and_cost.reduce(lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2]))
        # Update weights and bias
        w -= learning_rate * ((dw / m) + lambda_reg * w)
        b -= learning_rate * db / m

        reg_cost = (lambda_reg / (2 * feature_count)) * np.sum(np.square(w))
        cost = (total_cost / m) + reg_cost
        print(f"Iteration {n+1}/{iterations} - Cost: {cost}")

        bc_w.unpersist()
        bc_b.unpersist()

    return w, b


In [8]:

# Now use this smaller RDD for training
w, b = train(normalized_RDD, 10, 1.5, 0)


Iteration 1/10 - Cost: 1.4998030671352984


Iteration 2/10 - Cost: 0.7452491273041107


Iteration 3/10 - Cost: 0.4463838653243955


Iteration 4/10 - Cost: 0.33598210709531867


Iteration 5/10 - Cost: 0.2858186861976214


Iteration 6/10 - Cost: 0.2581741737711223


Iteration 7/10 - Cost: 0.24076133573757508


Iteration 8/10 - Cost: 0.22876056610476408


Iteration 9/10 - Cost: 0.21995387544017125


Iteration 10/10 - Cost: 0.21318893422096608


In [9]:
print(w[0])
print(b)
print(w.shape)



-0.19840144016729716
0.016435331016430403
(11,)


In [10]:
def predict(w, b, x):
    z = 0
    for j in range(len(w)):
        z += w[j] * x[j]

    z+= b
    # Compute the sigmoid of z
    y_hat = sigmoid(z)

    if y_hat > 0.5:
        return 1
    else: 
        return 0




In [11]:
def accuracy(w, b, RDD_Xy):
    
    results = RDD_Xy.map(lambda r: 1 if predict(w, b, r[0]) == r[1] else 0)
    results = results.reduce( lambda a, b: a + b)

    accuracy = results / RDD_Xy.count()
    return accuracy




In [12]:
acc = accuracy(w, b, normalized_RDD)
print("Accuracy: ",acc )

Accuracy:  0.930181


In [13]:

# Additional function to assign fold indices
def assign_fold_index(RDD_Xy, num_folds):
    num_samples = RDD_Xy.count()
    fold_sizes = (num_samples // num_folds) * np.ones(num_folds, dtype=int)
    fold_sizes[:num_samples % num_folds] += 1
    fold_indices = np.hstack([np.full(fold_size, fold_index) for fold_index, fold_size in enumerate(fold_sizes)])
    np.random.shuffle(fold_indices)
    return RDD_Xy.zipWithIndex().map(lambda x: (fold_indices[x[1]], x[0]))

# Cross-validation function
def cross_validate(RDD_Xy, num_folds, iterations, learning_rate, lambda_reg):
    # Assign each row to a fold
    indexed_RDD = assign_fold_index(RDD_Xy, num_folds)

    # Broadcast the indexed RDD
    bc_indexed_RDD = sc.broadcast(indexed_RDD.collect())

    # Initialize array to hold accuracy for each fold
    fold_accuracies = []

    # Perform cross-validation
    for fold in range(num_folds):
        train_RDD = sc.parallelize([x[1] for x in bc_indexed_RDD.value if x[0] != fold])
        test_RDD = sc.parallelize([x[1] for x in bc_indexed_RDD.value if x[0] == fold])
        
        # Train the model on the training set
        w, b = train(train_RDD, iterations, learning_rate, lambda_reg)

        # Evaluate the model on the validation set
        fold_acc = accuracy(w, b, test_RDD)
        fold_accuracies.append(fold_acc)
        print(f"Fold {fold+1}/{num_folds} - Accuracy: {fold_acc}")

    # Calculate average accuracy across all folds
    avg_acc = sum(fold_accuracies) / num_folds
    return avg_acc



In [14]:
# Call the cross-validation function
avg_accuracy = cross_validate(normalized_RDD, 10, 10, 1.5, 0)
print("Average Accuracy: ", avg_accuracy)


Iteration 1/10 - Cost: 1.499794021656618


Iteration 2/10 - Cost: 0.7451735280730557


Iteration 3/10 - Cost: 0.44635985587310784


Iteration 4/10 - Cost: 0.3359381192787715


Iteration 5/10 - Cost: 0.28577209507387863


...

Iteration 9/10 - Cost: 0.21974068553895312


Iteration 10/10 - Cost: 0.21296801183412686
Fold 6/10 - Accuracy: 0.92856


Iteration 1/10 - Cost: 1.4999729194723457


23/12/06 17:07:35 ERROR Inbox: An error happened while processing message in the inbox for LocalSchedulerBackendEndpoint
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3537)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:100)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:130)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1896)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1805)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1202)
	at java.base/java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:354)
	at org.apache.spark.serializer.JavaSerializationStream.writeObject(JavaSerializer.scala:46)
	at org.apache.spark.serializer.JavaSerializerInst

KeyboardInterrupt: 

In [ ]:

import numpy as np
import random

def assign_random_indices(RDD):
    total_count = RDD.count()
    random_indices = RDD.context.parallelize(range(total_count), RDD.getNumPartitions())\
                                .map(lambda x: random.randint(0, 9))
    return RDD.zip(random_indices)

def get_block_data(RDD, block_index):
    # Use flatMap to create training and test sets
    def split_data(item):
        (record, index) = item
        if index == block_index:
            return [(record, 'test')]
        else:
            return [(record, 'train')]

    split_RDD = RDD.flatMap(split_data)
    # Separate training and test data
    tr_data = split_RDD.flatMap(lambda x: [x[0]] if x[1] == 'train' else [])
    test_data = split_RDD.flatMap(lambda x: [x[0]] if x[1] == 'test' else [])
    return tr_data, test_data


def cross_validation_hyperparameter_tuning(RDD, learning_rates, lambda_values, iteration_counts):
    best_params = {'learning_rate': 0, 'lambda_reg': 0, 'iterations': 0, 'accuracy': 0}
    num_blocks_cv = 10

    RDD_with_indices = assign_random_indices(RDD)

    for learning_rate in learning_rates:
        for lambda_reg in lambda_values:
            for iterations in iteration_counts:
                accuracies = []

                for i in range(num_blocks_cv):
                    tr_data, test_data = get_block_data(RDD_with_indices, i)
                    params = train(tr_data, iterations, learning_rate, lambda_reg)
                    w, b = params[:-1], params[-1]
                    acc = accuracy(w, b, test_data)
                    accuracies.append(acc)

                avg_acc = sum(accuracies) / num_blocks_cv
                if avg_acc > best_params['accuracy']:
                    best_params['accuracy'] = avg_acc
                    best_params['learning_rate'] = learning_rate
                    best_params['lambda_reg'] = lambda_reg
                    best_params['iterations'] = iterations

    return best_params

# Hyperparameter ranges
learning_rates = [0.1, 1, 1.5]
lambda_values = [0.01, 0.1, 1]
iteration_counts = [10]

# Perform cross-validation with hyperparameter tuning
RDD_Xy = readFile('data/botnet_tot_syn_l.csv')
normalized_RDD = normalize(RDD_Xy).cache()
best_hyperparameters = cross_validation_hyperparameter_tuning(normalized_RDD, learning_rates, lambda_values, iteration_counts)

print("Best Hyperparameters:")
print("Learning Rate:", best_hyperparameters['learning_rate'])
print("Lambda:", best_hyperparameters['lambda_reg'])
print("Iterations:", best_hyperparameters['iterations'])
print("Accuracy:", best_hyperparameters['accuracy'])


23/12/06 16:57:00 WARN BlockManager: Task 201 already completed, not releasing lock for rdd_29_0
Traceback (most recent call last):
  File "/Users/francescomattioli/opt/miniconda3/envs/MPML/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 193, in manager
  File "/Users/francescomattioli/opt/miniconda3/envs/MPML/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/francescomattioli/opt/miniconda3/envs/MPML/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1291, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/francescomattioli/opt/miniconda3/envs/MPML/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError
23/12/06 16:57:01 ERROR Executor: Exception in task 5.0 in stage 26.0 (TID 207)
org.apache.spark.SparkException: Can only zip RDDs with same number o

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 202) (192.168.1.181 executor driver): org.apache.spark.SparkException: Can only zip RDDs with same number of elements in each partition
	at org.apache.spark.errors.SparkCoreErrors$.canOnlyZipRDDsWithSamePartitionSizeError(SparkCoreErrors.scala:142)
	at org.apache.spark.rdd.RDD$$anon$3.hasNext(RDD.scala:971)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.rdd.RDD$$anon$3.foreach(RDD.scala:967)
	at org.apache.spark.api.python.PythonRDD$.writeIteratorToStream(PythonRDD.scala:322)
	at org.apache.spark.api.python.PythonRunner$$anon$2.writeIteratorToStream(PythonRunner.scala:751)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.$an